In [51]:
import json
from datetime import datetime, timedelta
import os 
import time

import plotly

from pyspark.sql import functions as F
from pyspark.sql.window import Window

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import *

import cx_Oracle

import pandas as pd
import numpy as np
from builtins import min, round

def get_ora_conn():
    # The file contains your Oracle Account
    with open("notebooks/apps/vn.bdp.crm/duy_test/tasks/plotlyresearch/finalize/config.json", "r") as f:
            config = json.load(f)


    email = config["email"]
    ora_conn = config["ora_conn"] 
    ora_user = config["ora_user"] 
    ora_password = config["ora_password"]
    host = config["host"] 
    port = config["port"] 
    service_name = config["service_name"] 

    # Oracle Session
    dsn_tns = cx_Oracle.makedsn(host, port, service_name= service_name)
    conn = cx_Oracle.connect(user=ora_user, password=ora_password, dsn=dsn_tns)
    cursor = conn.cursor()
    
    return conn

conn = get_ora_conn()

In [25]:
df = pd.read_sql("select * from ap_crm.test_customer_flow_forward_052021", con = conn)

In [26]:
df.to_csv("notebooks/apps/vn.bdp.crm/duy_test/tasks/plotlyresearch/finalize/testdata.csv", index = 0)

In [27]:
df = pd.read_csv("notebooks/apps/vn.bdp.crm/duy_test/tasks/plotlyresearch/finalize/testdata.csv")

In [61]:
dfd = df[df['INI_SIGN_MONTH'].isin(["2021-06-01"])].iloc[:100,:]

In [62]:
# RENAME AND GET UINQUE NODES
dfd['INI_CONTRACT_TYPE'] = dfd['INI_CONTRACT_TYPE'].apply(lambda x: f"INIT_{x}")
dfd['CONTRACT1_TYPE'] = dfd['CONTRACT1_TYPE'].apply(lambda x: f"C1_{x}")
dfd['CONTRACT2_TYPE'] = dfd['CONTRACT2_TYPE'].apply(lambda x: f"C2_{x}")
dfd['CONTRACT3_TYPE'] = dfd['CONTRACT3_TYPE'].apply(lambda x: f"C3_{x}")

In [65]:
node_col1 = 'CONTRACT1_TYPE'
node_col2 = 'CONTRACT2_TYPE'
val_col = 'CLIENTS'
val_agg = 'sum'

df2 = dfd.groupby([node_col1, node_col2], as_index=0).agg(vals = (val_col, val_agg))
df2[f"{node_col1}_sum"] = df2.groupby(node_col1)['vals'].transform("sum")
df2[f"{node_col2}_sum"] = df2.groupby(node_col2)['vals'].transform("sum")

total_vals = df2['vals'].sum()

# get percentage on node
df2[f'%{node_col1}'] = df2[f'{node_col1}_sum'].apply(lambda x: x * 100/total_vals)
df2[f'%{node_col2}'] = df2[f"{node_col2}_sum"].apply(lambda x: x * 100/total_vals)

# get percentage on link
df2[f"%{node_col2}/{node_col1}"] = round((df2['vals'] / df2[f"{node_col1}_sum"]) * 100)

In [66]:
df2

,CONTRACT1_TYPE,CONTRACT2_TYPE,vals,CONTRACT1_TYPE_sum,CONTRACT2_TYPE_sum,%CONTRACT1_TYPE,%CONTRACT2_TYPE,%CONTRACT2_TYPE/CONTRACT1_TYPE
0,C1_CC,C2_CD,4,94,300,2.614743,8.344924,4.0
1,C1_CC,C2_CL,4,94,22,2.614743,0.611961,4.0
2,C1_CC,C2_NC,86,94,3242,2.614743,90.180807,91.0
3,C1_CD,C2_CC,22,3278,26,91.182197,0.723227,1.0
4,C1_CD,C2_CD,286,3278,300,91.182197,8.344924,9.0
5,C1_CD,C2_CL,10,3278,22,91.182197,0.611961,0.0
6,C1_CD,C2_HPL,1,3278,1,91.182197,0.027816,0.0
7,C1_CD,C2_NC,2958,3278,3242,91.182197,90.180807,90.0
8,C1_CD,C2_TW,1,3278,4,91.182197,0.111266,0.0
9,C1_CL,C2_CC,3,195,26,5.424200,0.723227,2.0
